In [1]:
import numpy as np

In [7]:
class NeuroPop:
    """
    This class implements several conveniences for 
    plotting, fitting and decoding from population tuning curves
    
    We assume that neurons have a tuning curve of the form:
    f(x) = b_ + g_ * exp(k_ * cos(x - mu_))
    
    Parameters
    ----------
    x: float, n_samples x 1, feature of interest
    Y: float, n_samples x n_neurons, population activity
    mu_: float,  n_neurons x 1, preferred feature [-pi, pi]
    k_: float,  n_neurons x 1, shape (width)
    g_: float,  n_neurons x 1, gain
    b_: float,  n_neurons x 1, baseline
    
    Methods
    -------
    encode
    tunefit
    display
    decode

    Other helper methods
    --------------------
    vonmises
    loss
    grad_loss
    jointlogL    
    """
    
    def __init__(self, x, Y):
        """
        Initialize the object
        """
        
        self.x = x
        self.Y = Y
        (n_samples, n_neurons) = Y.shape
        self.n_neurons = n_neurons
        # Assign random parameters
        
        self.mu_ = np.pi*(2.0*np.random.rand(n_neurons) - 1.0)
        self.k_ = 50.0*np.random.rand(n_neurons)
        self.g_ = 5.0*np.random.rand(n_neurons)
        self.b_ = 10.0*np.random.rand(n_neurons)
        
    #-----------------------------------------------------------------------
    def vonmises(x, mu, k, g, b):
        """
        The von Mises tuning function
        
        Parameters
        ----------
        x: float, n_samples x 1, feature of interest
        mu: float,  n_neurons x 1, preferred feature [-pi, pi]
        k: float,  n_neurons x 1, shape (width)
        g: float,  n_neurons x 1, gain
        b: float,  n_neurons x 1, baseline
        
        Outputs
        -------
        Y: float, n_samples x 1, firing rates
        """
        Y = b + g * np.exp (k * cos(x - mu))
        return Y
    
    #-----------------------------------------------------------------------
    def loss(x, y, mu, k, g, b):
        """
        The loss function: negative Poisson log likelihood function 
        under the von mises tuning model
        
        Parameters
        ----------
        x: float, n_samples x 1, feature of interest
        y: float, n_samples x 1, firing rates
        mu: float,  n_neurons x 1, preferred feature [-pi, pi]
        k: float,  n_neurons x 1, shape (width)
        g: float,  n_neurons x 1, gain
        b: float,  n_neurons x 1, baseline
        
        Outputs
        -------
        loss: float, scalar
        """
        lmb = b + g * np.exp (k * cos(x - mu))
        loss = np.sum(lmb) - np.sum(y * lmb)
        return loss
    
    #-----------------------------------------------------------------------
    def grad_loss(x, y, mu, k, g, b):
        """
        The gradient of the loss function:
        wrt parameters of the von mises tuning model
        
        Parameters
        ----------
        x: float, n_samples x 1, feature of interest
        y: float, n_samples x 1, firing rates
        mu: float,  n_neurons x 1, preferred feature [-pi, pi]
        k: float,  n_neurons x 1, shape (width)
        g: float,  n_neurons x 1, gain
        b: float,  n_neurons x 1, baseline
        
        Outputs
        -------
        grad_mu: float, scalar
        grad_k: float, scalar
        grad_g: float, scalar
        grad_b: float, scalar
        """
        
        lmb = b + g * np.exp (k * cos(x - mu))
        grad_mu = np.sum(g * np.exp(k * np.cos(x - mu)) * k * np.sin(x - mu) * (1 - y/lmb))
        grad_k = np.sum(g * np.exp(k * np.cos(x - mu)) * np.cos(x - mu) * (1 - y/lmb))
        grad_g = np.sum(g * np.exp(k * np.cos(x - mu)) * (1 - y/lmb))
        grad_b = np.sum((1-y/lmb))
        return grad_mu, grad_k, grad_g, grad_b
    
    #-----------------------------------------------------------------------
    def encode(self, x):
        """
        Compute the firing rates for the population 
        based on the von Mises tuning models
        given features
        
        Parameters
        ----------
        x: float, n_samples x 1, feature of interest
        
        Outputs
        -------
        Y: float, n_samples x n_neurons, population activity
        """
        # For each neuron
        for n in len(self.n_neurons):
            # Compute the firing rate under the von Mises model
            Y[:, n] = vonmises(x, self.mu_[n], self.k_[n], self.g_[n], self.b_[n])
        return Y
    
    #-----------------------------------------------------------------------
    def tunefit(self):
        """
        Estimate the parameters of the tuning curve under the 
        von Mises model, given features and population activity
        """
    
    #-----------------------------------------------------------------------
    def decode(self):
        """
        Estimate the parameters of the tuning curve under the 
        von Mises model, given features and population activity
        """
        
    #-----------------------------------------------------------------------
    def display(self):
        """
        Estimate the parameters of the tuning curve under the 
        von Mises model, given features and population activity
        """
        #for 

## Fitting tuning curves with gradient descent

The firing rates $y$ can be modeled as a Poisson random variable. 

$$
y = \text{Poisson}(\lambda)
$$

The mean $\lambda$ is given by the von Mises tuning model as follows.

$$
\lambda = b + g\exp\Big(\kappa \cos(x - \mu)\Big)
$$

Given a set of observations $(x_i, y_i)$, to identify the parameters $\Theta = \left\{\mu, \kappa, g, b\right\}$ we use gradient descent on the loss function $J$, specified by the negative log-likelihood,

$$
J = -\log\mathcal{L} = \sum_{i} \lambda_i - y_i \log \lambda_i
$$

Taking the gradients, we get:

$$
\frac{\partial J}{\partial \mu} = \sum_{i} g \exp\Big(\kappa \cos(x_i - \mu)\Big) \kappa \sin(x_i - \mu)\bigg(1 - \frac{y_i}{\lambda_i}\bigg)
$$

$$
\frac{\partial J}{\partial \kappa} = \sum_{i} g \exp\Big(\kappa \cos(x_i - \mu)\Big) \cos(x_i - \mu)\bigg(1 - \frac{y_i}{\lambda_i}\bigg)
$$

$$
\frac{\partial J}{\partial g} = \sum_{i} g \exp\Big(\kappa \cos(x_i - \mu)\Big) \bigg(1 - \frac{y_i}{\lambda_i}\bigg)
$$

$$
\frac{\partial J}{\partial b} = \sum_{i} \bigg(1 - \frac{y_i}{\lambda_i}\bigg)
$$

In [5]:
a = np.array([12.1, 3.1])
b = np.array([2.0, 21.1])
a/b

array([ 6.05      ,  0.14691943])